In [13]:
from datasets import load_dataset
from transformers import AutoFeatureExtractor


None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


## Data Preparation
### Load Dataset

In [14]:
gtzan = load_dataset("marsyas/gtzan", "all")

In [15]:
# Train Test Split
gtzan = gtzan["train"].train_test_split(seed=42, shuffle=True, test_size=0.1)

In [16]:
gtzan

DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'genre'],
        num_rows: 899
    })
    test: Dataset({
        features: ['file', 'audio', 'genre'],
        num_rows: 100
    })
})

### Feature Extraction